In [2]:
import pdfplumber
import pandas as pd

pdf_path = "../pdfs/relatorio-anual-de-gestao-2024-rag.pdf"
numero_da_pagina_indice = 18

with pdfplumber.open(pdf_path) as pdf:
    pagina = pdf.pages[numero_da_pagina_indice]
    tabela_bruta = pagina.extract_table()

df = pd.DataFrame(tabela_bruta)
#df.head()

In [3]:
#se a primeira celula da lista gerada pelo pdfplumber n for Nº nem null guarda no df pois deve ser a meta
df_metas = df[df[0].notna() & (df[0] != 'Nº')].copy()

colunas_metas = {
    0: 'Nº',
    2: 'Descrição da Meta',
    4: 'Indicador',
    6: 'Meta Prevista',
    8: '1º Trimestre',
    10: '2º Trimestre',
    12: '3º Trimestre',
    14: 'Meta Anual 2024'
}

df_metas_limpo = df_metas[colunas_metas.keys()].copy().reset_index(drop=True)
df_metas_limpo = df_metas_limpo.rename(columns=colunas_metas)
df_metas_limpo = df_metas_limpo.fillna('')

for coluna in df_metas_limpo.columns:
    if df_metas_limpo[coluna].dtype == 'object':
        df_metas_limpo[coluna] = df_metas_limpo[coluna].str.replace('\n', ' ', regex=False)

display(df_metas_limpo)

,Nº,Descrição da Meta,Indicador,Meta Prevista,1º Trimestre,2º Trimestre,3º Trimestre,Meta Anual 2024
0,1,Ampliar a atuação clínico-assistencial dos pro...,Número de protocolos instituídos a nível munic...,01,00,00,01,01
1,2,Ampliar o acesso aos serviços de saúde por mei...,Número de unidades de saúde com horário estend...,01,14,18,19,18


In [ ]:
df_acoes = df[df[6] == 'Ação realizada:'].copy()
df_descricoes = df[(df.notna().sum(axis=1) == 1) & (df[6].notna())].copy()

colunas_acoes = {
    3: 'Descrição da Ação',
    9: 'Marcação SIM',
    12: 'Marcação NÃO',
    14: 'Marcação PARCIAL'
}

df_acao_limpo = df_acoes[colunas_acoes.keys()].copy().reset_index(drop=True)
df_acao_limpo = df_acao_limpo.rename(columns=colunas_acoes)
df_acao_limpo = df_acao_limpo.fillna('')

df_acao_limpo['Monitoramento acao'] = df_descricoes[6].values

for coluna in df_acao_limpo.columns:
    if df_acao_limpo[coluna].dtype == 'object':
        df_acao_limpo[coluna] = df_acao_limpo[coluna].str.replace('\n', ' ', regex=False).str.strip()

display(df_acao_limpo)

,Descrição da Ação,Marcação SIM,Marcação NÃO,Marcação PARCIAL,Monitoramento acao
0,Implementar o protocolo de acolhimento na aten...,,x,,"Protocolo não implementado, membros do GT Acol..."
1,Instituir turno estendido em pelo menos uma Un...,x,,,"1 ESF Roberto Binato, 8 ESF Maringá,2 EAP Walt..."
